In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [10]:
engine = create_engine('mysql+pymysql://root:@127.0.0.1:3306/db')
flights = pd.read_sql_table('Flights', engine)
flight_arrivals = pd.read_sql_table('FlightArrivals', engine)
drivers = pd.read_sql_table('Drivers', engine)
tasks = pd.read_sql_table('Tasks', engine)

In [11]:
flight_arrivals.head(1)

,arrival_id,flight_id,aibt,trip_time,earliest_pickup_minutes,first_bag_on_belt_minutes
0,1,SQ391,2023-09-16 05:08:00,00:02:37,11,15


In [16]:
def get_available_driver():
    assigned_drivers = tasks[tasks['is_completed'] == 0]['driver_id'].unique()
    available_driver = drivers[~drivers['driver_id'].isin(assigned_drivers)].iloc[0]
    return available_driver

In [20]:
def create_first_task_for_flight_arrival(flight_arrival):
    task = {
        'arrival_id': flight_arrival['arrival_id'],
        'driver_id': get_available_driver()['driver_id'],
        'is_completed': 0,
        'task_id': 1,
    }
    tasks = tasks.append(task, ignore_index=True)
    task_df = pd.DataFrame([task])
    task_df.to_sql('Tasks', con=engine, if_exists='append', index=False)
    return task

In [ ]:
def start_task(task_id):
    task = tasks[tasks['task_id'] == task_id].iloc[0]
    task['is_completed'] = 1
    task['start_time'] = pd.Timestamp.now()
    tasks.update(task)
    tasks.to_sql('Tasks', con=engine, if_exists='append', index=False)
    return task

In [21]:
def end_task(task_id):
    task = tasks[tasks['task_id'] == task_id].iloc[0]
    task['is_completed'] = 1
    task['end_time'] = pd.Timestamp.now()
    tasks.update(task)
    tasks.to_sql('Tasks', con=engine, if_exists='append', index=False)
    return task

In [ ]:
create_first_task_for_flight_arrival(flight_arrivals.loc[0,:])


{'arrival_id': np.int64(1),
 'driver_id': 'Driver1',
 'is_completed': 0,
 'task_number': 1}

In [ ]:
def mark_as_completed(task_id):
    tasks.loc[tasks['task_id'] == task_id, 'is_completed'] = 1